# 🎯 HPC Benchmark Results Visualization

Interactive analysis and visualization of Globtim HPC benchmark results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 HPC Results Visualization Notebook")
print("=" * 40)

## 📥 Load Data

In [ ]:
# Load CSV data
df = pd.read_csv('hpc_results/benchmark_results.csv')
print(f"✅ Loaded {len(df)} benchmark results")

# Load detailed JSON data
with open('hpc_results/collection_summary.json', 'r') as f:
    detailed_results = json.load(f)
    
print(f"✅ Loaded detailed results for {len(detailed_results)} jobs")

# Display basic info
print("\n📋 Data Overview:")
print(df.info())
print("\n📊 Success Summary:")
print(df['success'].value_counts())

## 🎯 Success Rate Analysis

In [ ]:
# Create success rate visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Success rate pie chart
success_counts = df['success'].value_counts()
ax1.pie(success_counts.values, labels=['Failed', 'Success'], autopct='%1.1f%%', 
        colors=['#ff6b6b', '#51cf66'])
ax1.set_title('Overall Success Rate')

# Success by function
success_by_func = df.groupby('function_name')['success'].agg(['count', 'sum'])
success_by_func['rate'] = success_by_func['sum'] / success_by_func['count']

ax2.bar(success_by_func.index, success_by_func['rate'], 
        color=['#ff6b6b' if x < 0.5 else '#51cf66' for x in success_by_func['rate']])
ax2.set_title('Success Rate by Function')
ax2.set_ylabel('Success Rate')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 🔍 Detailed Analysis of Successful Jobs

In [ ]:
# Extract detailed metrics from successful jobs
successful_jobs = []

for job_id, job_data in detailed_results.items():
    parsed = job_data.get('parsed_results', {})
    if parsed.get('success', False):
        job_metrics = {
            'job_id': job_id,
            'function_name': parsed.get('function_name', 'unknown'),
            'globtim_working': parsed.get('globtim_working', False),
            'test_suite_completed': parsed.get('test_suite_completed', False),
            '4d_min_value': parsed.get('4d_min_value'),
            '4d_max_value': parsed.get('4d_max_value'),
            '4d_mean_value': parsed.get('4d_mean_value'),
            '4d_best_value': parsed.get('4d_best_value'),
            '4d_distance_to_origin': parsed.get('4d_distance_to_origin'),
            'simple_min_value': parsed.get('simple_min_value'),
            'simple_max_value': parsed.get('simple_max_value'),
            'simple_mean_value': parsed.get('simple_mean_value'),
        }
        successful_jobs.append(job_metrics)

if successful_jobs:
    success_df = pd.DataFrame(successful_jobs)
    print(f"✅ Found {len(success_df)} successful jobs")
    print("\n📊 Successful Job Details:")
    display(success_df)
else:
    print("⚠️  No successful jobs found")

## 🎨 Interactive 4D Results Visualization

In [ ]:
if successful_jobs and any(job.get('4d_min_value') is not None for job in successful_jobs):
    # Create interactive 4D results plot
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('4D Value Distribution', '4D vs Simple Comparison', 
                       'Distance to Origin', 'Performance Summary'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "table"}]]
    )
    
    for job in successful_jobs:
        if job.get('4d_min_value') is not None:
            # 4D value distribution
            values_4d = [job['4d_min_value'], job['4d_mean_value'], job['4d_max_value']]
            fig.add_trace(
                go.Scatter(x=['Min', 'Mean', 'Max'], y=values_4d, 
                          mode='lines+markers', name=f"4D Job {job['job_id']}"),
                row=1, col=1
            )
            
            # Distance visualization
            if job.get('4d_distance_to_origin') is not None:
                fig.add_trace(
                    go.Bar(x=[f"Job {job['job_id']}"], y=[job['4d_distance_to_origin']], 
                          name="Distance to Origin"),
                    row=2, col=1
                )
    
    # Add performance summary table
    if successful_jobs:
        job = successful_jobs[0]  # Show first successful job
        table_data = [
            ['Metric', 'Value'],
            ['Job ID', job['job_id']],
            ['Function', job['function_name']],
            ['4D Best Value', f"{job.get('4d_best_value', 'N/A'):.4f}" if job.get('4d_best_value') else 'N/A'],
            ['4D Distance', f"{job.get('4d_distance_to_origin', 'N/A'):.4f}" if job.get('4d_distance_to_origin') else 'N/A'],
            ['Simple Min', f"{job.get('simple_min_value', 'N/A'):.4f}" if job.get('simple_min_value') else 'N/A'],
            ['Globtim Working', '✅' if job.get('globtim_working') else '❌']
        ]
        
        fig.add_trace(
            go.Table(
                header=dict(values=table_data[0], fill_color='lightblue'),
                cells=dict(values=list(zip(*table_data[1:])), fill_color='white')
            ),
            row=2, col=2
        )
    
    fig.update_layout(height=600, title_text="🎯 HPC Benchmark Results Dashboard")
    fig.show()
    
else:
    print("⚠️  No 4D results available for visualization")

## 📈 Time Series Analysis

In [ ]:
# Convert collection times to datetime
df['collection_datetime'] = pd.to_datetime(df['collection_time'])

# Create time series plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plot success/failure over time
for success_val, color, label in [(True, 'green', 'Success'), (False, 'red', 'Failed')]:
    subset = df[df['success'] == success_val]
    if len(subset) > 0:
        ax.scatter(subset['collection_datetime'], subset['job_id'], 
                  c=color, label=label, alpha=0.7, s=100)

ax.set_xlabel('Collection Time')
ax.set_ylabel('Job ID')
ax.set_title('🕒 Job Success/Failure Timeline')
ax.legend()
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"📊 Timeline shows {len(df)} jobs collected over time")

## 🔧 Export Enhanced Visualizations

In [ ]:
# Save enhanced plots
output_dir = Path('hpc_results/visualizations')
output_dir.mkdir(exist_ok=True)

print(f"💾 Saving enhanced visualizations to {output_dir}/")
print("✅ Interactive notebook visualization complete!")
print("\n🎯 Next steps:")
print("1. Run more benchmark jobs to get richer data")
print("2. Use this notebook to analyze trends")
print("3. Export specific plots for presentations")